# 2MASX J15220246+5010185 - Image Reducer - No Pre CR Removal

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook should be run with the <span style="font-family: 'Ariel', monospace;">stenv</span> environment.
</div>

Instead of working with the DeepCR images, this one aligns the DRCs then uses TweakBack to propagate
those WCS solutions to the FLCs.

The purpose of this notebook is to reduce the FLC files from Hubble by:

1. Aligning FLCs to the GAIA catalog
2. Drizzling Images together from a particular filter

## Imports

In [ ]:
# Python Imports
import os
from pathlib import Path

# Astropy Colab Imports
from astropy.io import fits
from drizzlepac.tweakreg import TweakReg
from drizzlepac.tweakback import apply_tweak
from drizzlepac.astrodrizzle import AstroDrizzle

## Notebook Setup

In [ ]:
# Check Directory
if Path.cwd().name != "Images":
    if Path.cwd().name == "Notebooks":
        os.chdir("../Images")
    else:
        RuntimeError("This notebook must be run from the Images directory.")
print(f"Current Directory: {Path.cwd()}")

In [ ]:
# Data Directory
DATA_DIR = Path('mastDownload/HST')

# FLC Glob Pattern
FLC_CR_GLOB_PAT = '*[!crclean]_flc.fits'

# DRC Glob Pattern
DRC_CR_GLOB_PAT = '*_drc.fits'

## Load the Data

In [ ]:
# Get the File Names and Sort them by filter
flcNameDict = {}
for fn in DATA_DIR.rglob(FLC_CR_GLOB_PAT):

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in flcNameDict:
        flcNameDict[filt] = []
    flcNameDict[filt].append(fn)
flcNameDict

In [ ]:
# Get the File Names and Sort them by filter
drcNameDict = {}
for fn in DATA_DIR.rglob(DRC_CR_GLOB_PAT):

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in drcNameDict:
        drcNameDict[filt] = []
    drcNameDict[filt].append(fn)
drcNameDict

## Align Images to GAIA

Sometimes, the FLCs have so many CRs and so few Milky Way stars, it makes aligning the images difficult.
In this case, a different strategy is to align the Hubble Pipeline DRCs/DRZs to GAIA using `TweakReg` then
assigning the found WCS solution to the associated FLCs/FLTs using `TweakBack`.

For an example of this strategy, consider the work documented in the [Abell 1367](https://github.com/wwaldron/a1367)
repo on GitHub where the [Image Reducer Notebook](https://github.com/wwaldron/a1367/blob/main/Images/ImageReducer.ipynb)
implements this methodology.

<div class="alert alert-block alert-info">
    <b>Note:</b> In the cells below where <span style="font-family: 'Ariel', monospace;">TweakReg</span> is called,
    the user <i>must</i> update the <span style="font-family: 'Ariel', monospace;">updatehdr</span>
    keyword to <span style="font-family: 'Ariel', monospace;">True</span> and rerun the cell once a valid
    WCS solution is found.
    If the value is left as <span style="font-family: 'Ariel', monospace;">False</span>, the header in the
    input file will not be updated.
</div>

### Align F814W Image to GAIA

In [ ]:
# Image Dep Search Parameters
threshold  = 8
conv_width = 3.5

# Image Find Parameters
imagefindcfg = dict(
    threshold=threshold,
    conv_width=conv_width
)

# Align the First F814W Image to the GAIA data
TweakReg(
    str(drcNameDict['F814W'][0]),
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refcat='../Data/GAIA/2MASXJ15220246+5010185-GAIA-RefCatalog-icrs.txt',
    runfile=f'TweakReg-GAIA-F814W-DRC.log',
    searchrad=0.25,
    minobj=5,
    tolerance=1.5,
    imagefindcfg=imagefindcfg,
    interactive=False
)

### Align F475W Images to F814W

In [ ]:
# Image Find Parameters
refimagefindcfg = dict(
    # peakmax=900,
    threshold=threshold,
    conv_width=conv_width
)

# Align the First F475W Image to the GAIA data
TweakReg(
    str(drcNameDict['F475W'][0]),
    refimage=str(drcNameDict['F814W'][0]),
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    runfile=f'TweakReg-GAIA-F475W-DRC.log',
    searchrad=0.2,
    minobj=15,
    tolerance=0.75,
    imagefindcfg=imagefindcfg,
    refimagefindcfg=refimagefindcfg,
    interactive=False
)

### Align F275W Images to F814W

In [ ]:
# Image Dep Search Parameters
threshold  = 5.0
conv_width = 3.5

# Image Find Parameters
imagefindcfg = dict(
    # peakmax=900,
    threshold=threshold,
    conv_width=conv_width
)

# Align the First F275W Image to the GAIA data
TweakReg(
    str(drcNameDict['F275W'][0]),
    refimage=str(drcNameDict['F814W'][0]),
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    runfile=f'TweakReg-GAIA-F275W-DRC.log',
    searchrad=0.45,
    minobj=10,
    tolerance=1.50,
    imagefindcfg=imagefindcfg,
    refimagefindcfg=refimagefindcfg,
    interactive=False
)

In [ ]:
%%bash
# Move Log Files
mkdir -p logs/tweakreg
mv *.log logs/tweakreg

# Remove Intermediate Files
rm *.coo *.png

### TweakBack Solutions

In [ ]:
# Loop through DRCs
for fileList in drcNameDict.values():

    # Loop through the files
    for fn in fileList:

        # Get ASN
        asnName = fn.with_stem(fn.stem.replace('_drc', '_asn'))

        # Get the Input Dither Files
        flcFiles = fits.getdata(asnName, 'ASN')
        flcFiles = flcFiles['MEMNAME'][flcFiles['MEMTYPE'] == 'EXP-DTH']
        flcFiles = [DATA_DIR / flcName.lower() / f'{flcName.lower()}_flc.fits' for flcName in flcFiles]

        # Run Tweakback
        apply_tweak(
            fn.as_posix() + '[sci,1]', None, orig_wcs_key='A',
            input_files=','.join([flcName.as_posix() for flcName in flcFiles]),
            tweaked_wcs_name='GAIA'
        )

## Drizzle Images for CR Correction

Although there will be additional notes added later, it is worth noting that according to
[STScI](https://hst-docs.stsci.edu/drizzpac/chapter-6-reprocessing-with-the-drizzlepac-package/6-3-running-astrodrizzle#id-6.3RunningAstroDrizzle-SelectingtheOptimalScaleandPixfrac):

1. For sub-pixel dithered data, select an output scale that's smaller than the native scale.
It will even help in the cosmic ray rejection step.
1. A smaller final_pixfrac gives higher resolution and lower correlated noise, but also reduces
sensitivity to low-surface brightness features (though it is possible to convolve a high resolution
image later to go after low surface brightness features).
1. Keep the standard deviation of the weight map over the main part of the image to above ~0.3 of
the mean to insure that one does not lose significant signal-to-noise in ignoring the weight map in
final photometry.

To summarize the last step, a `final_scale`/`final_pixfrac` combo should be chosen such that,
for the weight image,
\begin{equation}
    \mathrm{std} \gtrsim 0.3 \, \mathrm{mean}
\end{equation}

### AstroDrizzle ACS Data Quality Flags

* [ACS DQ Flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)
* [WFC3-UVIS DQ Flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-2-uvis-data-calibration-steps#id-3.2UVISDataCalibrationSteps-3.2.3DataQualityArrayInitialization)
* [WFC3-IR DQ Flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-3-ir-data-calibration-steps#id-3.3IRDataCalibrationSteps-3.3.1DataQualityInitialization)

In [ ]:
# DQ Bits
DQ_WARM_PIX = 64
DQ_BAD_COL  = 128
DQ_FULL_WELL= 256
DQ_SINK_PIX = 1024
DQ_GOOD_PIX = DQ_WARM_PIX + DQ_BAD_COL + DQ_FULL_WELL + DQ_SINK_PIX # Make these OK

### Drizzle F814W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    [str(fn) for fn in flcNameDict['F814W']],
    output='SBS1520+503-F814W',
    runfile='F814W-Astro-NoPreCR.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=True,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    combine_nsigma='3 2.5',         # Default: 4.0 3.0
    driz_cr=True,
    driz_cr_snr='2.0 1.75',         # Default: 3.5 3.0
    driz_cr_grow=2,
    driz_cr_ctegrow=5,
    driz_cr_scale='1.2 0.7',        # Default: 1.2 0.7
    final_wht_type='IVM',
    final_kernel='square',
    final_pixfrac=1,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_rot=0,
    final_scale=0.03
)

### Drizzle F475W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    [str(fn) for fn in flcNameDict['F475W']],
    output='SBS1520+503-F475W',
    runfile='F475W-Astro-NoPreCR.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=True,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    combine_nsigma='3 2.5',         # Default: 4.0 3.0
    driz_cr=True,
    driz_cr_snr='2.0 1.75',         # Default: 3.5 3.0
    driz_cr_grow=2,
    driz_cr_ctegrow=5,
    driz_cr_scale='1.2 0.7',        # Default: 1.2 0.7
    final_wht_type='IVM',
    final_kernel='square',
    final_pixfrac=1,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_refimage='SBS1520+503-F814W_drc.fits'
)

### Drizzle F275W Images

In [ ]:
# DQ Bits
DQ_WARM_PIX = 64
DQ_FULL_WELL= 256
DQ_SINK_PIX = 1024
DQ_GOOD_PIX_WFC3 = DQ_WARM_PIX + DQ_FULL_WELL + DQ_SINK_PIX # Make these OK

In [ ]:
# Drizzle Images
AstroDrizzle(
    [str(fn) for fn in flcNameDict['F275W']],
    output='SBS1520+503-F275W',
    runfile='F275W-Astro-NoPreCR.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=True,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX_WFC3,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.5,
    final_bits=DQ_GOOD_PIX_WFC3,
    final_wcs=True,
    final_refimage='SBS1520+503-F814W_drc.fits'
)

In [ ]:
%%bash
# Move Log Files
mkdir -p logs/astrodrizzle
mv *.log logs/astrodrizzle

# Move Final Drizzled Images
mkdir -p ProcessedImages/HST/Drizzled
mv *_dr?.fits ProcessedImages/HST/Drizzled/